In [2]:
%pip install pokerkit

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from pokerkit import *

In [4]:
from random import sample
from itertools import product, combinations
import copy


In [5]:
from concurrent.futures import ProcessPoolExecutor

def oddsCalc(cards, board): #'AsKs', 'Kc8h8d'
    with ProcessPoolExecutor() as executor:
        return calculate_hand_strength(2, parse_range(cards),Card.parse(board), 2,  5, Deck.STANDARD, (StandardHighHand,), sample_count=2000, executor=executor,)

In [ ]:
oddsCalc('AsAd', 'AhAc2c2d6h')

1.0

In [46]:
def Nstreet(board):
    #returns number of street we at
    girth = len(board)/2
    return (girth>=3) + (girth>=4) + (girth==5)
    

In [ ]:
from math import *
def move(cards, board, stack, pot, aggressivity_index=1, odds_manual=None):
    #Allows custom odds to test
    if odds_manual != None:
        odds = odds_manual
    else:
        odds = oddsCalc(cards, board)
    print(odds)
    
    #ensures no division by 0 (if odds 50/50 (1-2*odds)=0)
    if (1-2*odds):
        tl = -pot + 2*stack*odds - stack
        tr = (pot**2 + 2*pot*stack + stack**2*(1-2*odds)**2)**0.5
        kelly_solutions = (tl +tr)/2, (tl-tr)/2
    else:
        kelly_solutions = -1, -1
    
    #finds max call+bet using kelly solutions
    callMax = (-(pot+stack-2*odds*stack)+((pot+stack-2*odds*stack)**2+4*odds*pot*stack)**0.5)/2  #, (-(pot+stack-2*odds*stack)-((pot+stack-2*odds*stack)**2+4*odds*pot*stack)**0.5)/2
    if kelly_solutions[0] + kelly_solutions[1] > 0:
        print(kelly_solutions)
        bet = round(kelly_solutions[0])
        # EV version: bet, callMax = round(kelly_solutions[0]), inf
    else:
        bet = 0
        # EV version: bet, callMax = 0, floor((odds*pot)/(1-2*odds))
    print(kelly_solutions)
    return aggressivity_index*bet, aggressivity_index*callMax/Nstreet(board) #is this the correct move? should i divide by the street? should i leave bet intact?


In [47]:
board = '9hTs9s'

Nstreet(board)


    

1

In [38]:
print(move('7c2s', '', 100, 50, .499999))
#something is broken

0.499999
(30.90159943758419, -80.90179943758417)
(0, 30.90159943758419)


In [ ]:
o = 0.001
p = 100
s = 500

b0, b1 = (-(p+s-2*o*s)+((p+s-2*o*s)**2+4*o*p*s)**0.5)/2, (-(p+s-2*o*s)-((p+s-2*o*s)**2+4*o*p*s)**0.5)/2
b0, b1

(0.08346082519307174, -599.083460825193)

In [58]:
import tkinter as tk
from tkinter import simpledialog, messagebox
from pokerkit import NoLimitTexasHoldem, Automation
import random

# --- Your bot logic ---
def move(hole, board, stack):
    return 50 if len(board) < 5 else 0  # Replace with real bot logic

class PokerGame:
    def __init__(self, root):
        self.root = root
        self.root.title("PokerKit GUI Poker")

        # PokerKit pre-defined game
        self.state = NoLimitTexasHoldem.create_state(
            (
                Automation.ANTE_POSTING,
                Automation.BET_COLLECTION,
                Automation.BLIND_OR_STRADDLE_POSTING,
                Automation.CARD_BURNING,
                Automation.HOLE_CARDS_SHOWING_OR_MUCKING,
                Automation.HAND_KILLING,
                Automation.CHIPS_PUSHING,
                Automation.CHIPS_PULLING,
            ),
            False,
            0,
            (5, 10),
            10,
            (1000, 1000),
            2,
        )

        self.setup_gui()
        self.state.deal_hole()
        self.update_gui()
        self.next_action()

    def setup_gui(self):
        self.status = tk.Label(self.root, text="", font=("Arial", 14))
        self.status.pack()

        self.hand_label = tk.Label(self.root, text="Your Hand: ", font=("Arial", 16))
        self.hand_label.pack()

        self.board_label = tk.Label(self.root, text="Board: ", font=("Arial", 16))
        self.board_label.pack()

        self.stack_label = tk.Label(self.root, text="", font=("Arial", 14))
        self.stack_label.pack()

        frame = tk.Frame(self.root)
        frame.pack(pady=10)

        self.btn_check = tk.Button(frame, text="Check", command=self.check)
        self.btn_bet = tk.Button(frame, text="Bet", command=self.bet)
        self.btn_call = tk.Button(frame, text="Call", command=self.call)
        self.btn_fold = tk.Button(frame, text="Fold", command=self.fold)

        self.btn_check.pack(side=tk.LEFT, padx=5)
        self.btn_bet.pack(side=tk.LEFT, padx=5)
        self.btn_call.pack(side=tk.LEFT, padx=5)
        self.btn_fold.pack(side=tk.LEFT, padx=5)

    def update_gui(self):
        board = ''.join(str(card) for card in self.state.board)
        hole = ''.join(str(card) for card in self.state.hole_cards[0])
        pot = self.state.pot
        stacks = self.state.stacks

        self.hand_label.config(text=f"Your Hand: {hole}")
        self.board_label.config(text=f"Board: {board}")
        self.stack_label.config(text=f"Pot: {pot} | You: {stacks[0]} | Bot: {stacks[1]}")
        self.status.config(text=f"{'Your' if self.state.next_player == 0 else 'Bot'} turn")

    def disable_all(self):
        for b in [self.btn_check, self.btn_bet, self.btn_call, self.btn_fold]:
            b.config(state=tk.DISABLED)

    def enable_buttons(self, actions):
        self.btn_check.config(state=tk.NORMAL if 'Check' in actions else tk.DISABLED)
        self.btn_bet.config(state=tk.NORMAL if 'CompleteBettingOrRaisingTo' in actions else tk.DISABLED)
        self.btn_call.config(state=tk.NORMAL if 'CheckOrCall' in actions else tk.DISABLED)
        self.btn_fold.config(state=tk.NORMAL if 'Fold' in actions else tk.DISABLED)

    def next_action(self):
        self.update_gui()

        if self.state.is_hand_over:
            winners = self.state.winners
            messagebox.showinfo("Hand Over", f"Winners: {winners}")
            self.state.start_hand()
            self.state.deal_hole()
            self.update_gui()
            self.next_action()
            return

        if self.state.next_player == 1:
            self.disable_all()
            self.root.after(500, self.bot_action)
        else:
            legal = {a.name for a in self.state.legal_actions}
            self.enable_buttons(legal)

    def check(self):
        self.state.check()
        self.next_action()

    def bet(self):
        max_bet = self.state.stacks[0]
        amount = simpledialog.askinteger("Bet", "How much?", minvalue=1, maxvalue=max_bet)
        if amount:
            self.state.complete_bet_or_raise_to(amount)
            self.next_action()

    def call(self):
        self.state.check_or_call()
        self.next_action()

    def fold(self):
        self.state.fold()
        self.next_action()

    def bot_action(self):
        hole = ''.join(str(c) for c in self.state.hole_cards[1])
        board = ''.join(str(c) for c in self.state.board)
        stack = self.state.stacks[1]
        amount = move(hole, board, stack)

        legal = {a.name for a in self.state.legal_actions}

        try:
            if amount == 0 and 'Check' in legal:
                self.state.check()
            elif amount > 0 and 'CompleteBettingOrRaisingTo' in legal:
                self.state.complete_bet_or_raise_to(amount)
            elif 'CheckOrCall' in legal:
                self.state.check_or_call()
            else:
                self.state.fold()
        except Exception as e:
            print(f"Bot error: {e}")
            self.state.fold()

        self.next_action()

# Launch GUI
root = tk.Tk()
app = PokerGame(root)
root.mainloop()

AttributeError: 'State' object has no attribute 'board'

In [ ]:
#add agressivity index
#include check percentage
#adjust to a per street basis cuz this is same as assuming opponent goes all in everytime